## LLM Agents Group Chat

*[Coding along with the Udemy online course AI Agents: Building Teams of LLM Agents that Work For You by Mohsen Hassan & Ilyass Tabiai]*

### Teamwork: Planning a stock report

What we're going to do in this exercise is to setup a team of agents and let them determine how to accomplish the task by themsleves. 

It's a less efficient method than the ones we've previously used and at the same time it's the one that requires the least of involvement on the user's side.

In order for the team to be able to work together, we will provide them with a __Planner agent__. The Planner agent's role will be 
- to determine which task should be done first
- and which agent should accomplish it.

This means that we should something to the agent's definitions to let other agents know about their role, so that the Planner can determine which agent can do which task.

__The task we are going to focus on will be writing a blogpost about the stock performance of a specific asset.__

In [2]:
import pandas as pd

api_key = pd.read_csv("~/tmp/chat_gpt/autogen_agent_1.txt", sep=" ", header=None)[0][0]

llm_config = {
    "model": "gpt-4o",
    "api_key": api_key
    }

print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


### Team description

* **User_proxy or Admin**: to allow the user to comment on the report and ask the writer to refine it.
* **Planner**: to determine relevant information needed to complete the task.
* **Engineer**: to write code using the defined plan by the planner.
* **Executor**: to execute the code written by the engineer.
* **Writer**: to write the report.

### The task

In [7]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-10-01."

## Defining our agents

In [4]:
import autogen

### User Proxy Agent

The __User Proxy Agent__ will allow us to interact with the agents. It can use auto-reply, so it will have __access to an LLM config__, but it will also have its __Human Input Mode to Always__, which will allow us to interact with the agents.

In [5]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config, 
    human_input_mode="ALWAYS", # If user does not provide feedback, the LLM will do it for us
)
user_proxy

[autogen.oai.client: 10-07 12:52:11] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Planner Agent

It is possible to add a **description message** to an agent, which is known by other agents and is meant to make tham understand this agent's role. This way other agents, and specifically the Planner, can determine which agent can accomplish which task.

The **system message prompt** that we used so far is *only known to each agent* and defines how they should behave.  

In [6]:
# defining a Planner Agent with a description
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

[autogen.oai.client: 10-07 13:08:54] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Engineer Agent

The Engineer agent already has a __default system prompt__, which is enough for the task we would like it to accomplish, write code.  
We are going to add a description message that lets other agents know about its role. Other agents will also know about the name of agents, and in somes cases that might be enough to define their roles.

In [8]:
# system msg: Already has a default one because it is a code writer
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

# Check engineer system prompt message
print(engineer.system_message)

[autogen.oai.client: 10-07 13:21:07] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, 

### Executor Agent

The Executor agent is a __code executor similar to the ones we already used__. Since this time there will be several agents exchanging through a group chat and we're unsure in which order that'll happen, we are going to add an argument in the `code_execution_config`, a parameter that specifies that __we want this agent to know about the last 3 messages instead of the last one only__.

__The name of the agent in this case is enough to share its role with other agents.__

In [9]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

### Writer Agent

This agent is the writer agent who is going to write the report we want as a blogpost. This agent will have a system message and a description.

In [10]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

[autogen.oai.client: 10-07 13:29:43] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### The Group Chat

We are now going to define the chat, this time, it's going to be a __Group Chat__. This means __we won't define ay conversation order__ other than specifying who is the Manager. We'll just list the agents who'll be taken part in this group chat, and the max number of turns  we allow for it to last:

In [11]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[], # No initial message
    max_round=20,
)

We're also going to provide this group chat with a __Group Chat Manager__.

In [12]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    llm_config=llm_config
)

[autogen.oai.client: 10-07 13:32:14] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Chat Initiation



Let's now initiate the chat with a message from the User Proxy agent (which is us) to the manager, giving him the task we previously defined:

In [17]:
'''
# code that would execute our task but it's not supposed to run inside a notebook :)
groupchat_result = user_proxy.initiate_chat(
    manager, # We initiate between user and manager so we can give the task
    message=task,
)
'''

"\n# code that would execute our task but it's not supposed to run inside a notebook :)\ngroupchat_result = user_proxy.initiate_chat(\n    manager, # We initiate between user and manager so we can give the task\n    message=task,\n)\n"

# Group Chat Result

__A first run of the Group Chat produced the following result:__

# Nvidia Stock Performance in September 2024: A Statistical Overview with Market Insights

As October rolls in, it's essential to look back and examine Nvidia's (NVDA) stock market performance over September 2024. This month saw vibrant market activity, marked by notable shifts and encouraging growth for one of the tech sector’s foremost innovators. Let's delve into Nvidia's market trends, supported by visual data and contextual analysis.

## Monthly Statistical Highlights

Starting at an opening price of $116.01 on September 1st, Nvidia's stock climbed to a closing price of $121.40 on September 30th, reflecting a moderate overall gain of around 4.65%. Here's a summary of significant monthly statistics:

- **Opening Price (Sep 1, 2024):** $116.01

- **Closing Price (Sep 30, 2024):** $121.40

- **Highest Price in September:** $127.67 (recorded on Sep 26)

- **Lowest Price in September:** $100.95 (recorded on Sep 6)

- **Average Closing Price:** $114.72<br/>

## Stock Price Visualization

Below is a plot reflecting Nvidia's daily closing prices throughout September 2024. The chart highlights the highest and lowest points of the month, providing a clear depiction of major price fluctuations and market trends.

![Nvidia Stock Price in September 2024](../coding/nvidia_stock_price_september_2024.png)

## Analyzing Key Market Movements

Significant price movements were observed on specific dates, driven by impactful news and market dynamics:

**Significant Rise on September 11:**
- Nvidia's stock saw an impressive spike of 8.15% on this date. This increase followed the announcement of key developments in their graphics card lineup, enhancing gaming computing capabilities, which generated widespread investor enthusiasm. According to *[TechCrunch]*, the announcement highlighted Nvidia's push into leveraging AI for graphics transformations, reinforcing market confidence.

**Sharp Decline on September 6:**
- Conversely, the steep price drop of 4.09% observed on September 6 can be linked to broader market anxieties over tech stock valuations following the latest US Federal Reserve interest rate decision. *[Bloomberg]* reported industry-wide effects that influenced sentiment across numerous tech stocks, including Nvidia.

## Conclusion and Forward Outlook

Nvidia demonstrated remarkable resilience amidst market fluctuations, closing September on a high note. The notable highs and lows present opportunities for agile investors and underscore Nvidia's dynamic market presence. Looking ahead, the focus will be on Nvidia’s strategic initiatives and sector movements. Expectations are high in anticipation of developments such as further advancements in AI and deep learning applications.

Stay tuned for our upcoming monthly analyses, where we'll continue to monitor Nvidia’s performance and dissect the factors shaping its market journey.


# Group Chat with constraints

In some cases, you might want to constraint exchanges within the chat. That measn that you might want to stop some agents from immediately interacting with other agents or the opposite. You can do so by adding an argument to your Group Chat definition.

For example, if we want to ensure that the engineer will only interact with the **Executor** or **User proxy** (us), we can specify that.  
Here is how we would define such constraints:

In [15]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=20,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

We can then simply re-start the chat with these constraints:

In [18]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

'''
# code that would execute our task but it's not supposed to run inside a notebook :)
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)
'''

[autogen.oai.client: 10-07 15:13:29] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


"\n# code that would execute our task but it's not supposed to run inside a notebook :)\ngroupchat_result = user_proxy.initiate_chat(\n    manager,\n    message=task,\n)\n"